In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
from tqdm import tqdm
from glob import glob
import os 
import matplotlib.pyplot as plt

<h3>the images must be a square images with 3 channels</h3>

In [4]:
GENERATE_SQUARE = 128
IMAGE_CHANNELS = 3
# Size of the vector to generate images from
SEED_SIZE = 128

In [5]:
# this way we can get the path to all the images
images_dir =  "../input/faces-data-new/images/"
images = glob(images_dir+'*.jpg')
images[:5]

<h3>here we need to read all the images with opencv and the reshape them to a square (96,96,3) and normelise them in the range [0,1] to use the sigmoid activation (we can also normelize between -1 and 1 and use tanh</h3>

In [6]:
training_data = []
for file in tqdm(images):
    image = cv2.imread(file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (128,128))
    training_data.append(image)
# here we normelize
training_data = np.array(training_data).astype(np.float32)
training_data = training_data / 255

In [7]:
training_data.shape

In [9]:
# Batch and shuffle the data this is a better way to use large datasets
train_dataset = tf.data.Dataset.from_tensor_slices(training_data).shuffle(5000).batch(128)

<h3>the gan architecture is consisted basically from 2 model generator that generates random images based on a randm seed vector and descriminator that predict if the image is real or generated we train them in an adversial way one by one so they can get better alongside and performe better because each model is trying to beat the other </h3>

In [10]:
# most of the hyper params are taken from the dcgan paper

def build_generator(seed_size):
    model = Sequential()

    model.add(Dense(4096, activation="relu", input_dim=seed_size))
    model.add(Reshape((4,4,256)))
    # output shape 4,4,256
    model.add(Conv2DTranspose(256, kernel_size=(3,3), strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    # output shape 8,8,256
    model.add(Conv2DTranspose(256, kernel_size=(3,3), strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    # output shape 16,16,256
    model.add(Conv2DTranspose(128, kernel_size=(3,3), strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    # output shape 32,32,128
    model.add(Conv2DTranspose(128, kernel_size=(3,3), strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    # output shape 64,64,128
    model.add(Conv2DTranspose(128, kernel_size=(3,3), strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    # output shape 128,128,128
    
    # Final CNN layer
    model.add(Conv2D(3, kernel_size=(3,3), padding="same"))
    model.add(Activation("sigmoid"))
    # output shape 128,128,3

    return model

In [11]:
def build_discriminator(image_shape):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    

    return model

In [13]:
generator = build_generator(SEED_SIZE)

seed = tf.random.normal([1, SEED_SIZE])
generated_image = generator(seed, training=False)

plt.imshow(generated_image[0, :, :, 0])

<h3>here both the generation and the decrimination are really bad because they are not trained yet</h3>

In [14]:
image_shape = (GENERATE_SQUARE,GENERATE_SQUARE,IMAGE_CHANNELS)

discriminator = build_discriminator(image_shape)
decision = discriminator(generated_image)
print (decision)

In [16]:
# here we define 2 functions to calculate the loss
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [17]:
generator_optimizer = tf.keras.optimizers.Adam(0.0001,0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0001,0.5)

In [18]:
# this is the way to train both of the models in a generative they must never train at the same time
@tf.function
def train_step(images):
    seed = tf.random.normal([128, SEED_SIZE])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(seed, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)


        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return gen_loss, disc_loss

In [19]:
def train(dataset, epochs):
    fixed_seed = np.random.normal(0, 1, (128 * 128, SEED_SIZE))
    
    for epoch in range(epochs):
        gen_loss_list = []
        disc_loss_list = []

        for image_batch in dataset:
            t = train_step(image_batch)
            gen_loss_list.append(t[0])
            disc_loss_list.append(t[1])

        g_loss = sum(gen_loss_list) / len(gen_loss_list)
        d_loss = sum(disc_loss_list) / len(disc_loss_list)

        print (f'Epoch {epoch+1}, gen loss={g_loss},disc loss={d_loss}')

In [21]:
train(train_dataset, 300)

<h3>here we train the model in adversial way </h3>

In [37]:
test = tf.random.normal([1, SEED_SIZE])
generated_image = generator(test)

plt.imshow(generated_image[0])

In [26]:
generator.save("face_generator.h5")
generator.save_weights("face_generator.weights.h5")